In [1]:
import sys
import os

project_root = os.path.abspath(os.getcwd())
if project_root not in sys.path:
    sys.path.append(project_root)
print(sys.path)

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/atupulazi/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/home/atupulazi/personal_projects/collision-forecast/src']


In [2]:
%pip install torch
%pip install torchaudio
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install scikit-image
%pip install scipy
%pip install tqdm

# %pip uninstall -y torch torchvision
# %pip install torch==2.7.1 torchvision==0.18.1
%pip install torch torchvision

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal sit

In [3]:
from model import build_videomae
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from dataloader import CollisionForecastDataset

/home/atupulazi/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def train_model(model, optimizer, criterion, device_param, train_loader, val_loader, FORCE_RETRAIN, epochs=10):
    model_name = "model"
    checkpoint_dir = "checkpoints"
    os.makedirs(checkpoint_dir, exist_ok=True)

    model_path = os.path.join(checkpoint_dir, f"{model_name}_best.pth")

    # Check if model already exists
    if not FORCE_RETRAIN and os.path.exists(model_path):
        print(f"Model {model_name} already exists. Skipping training.")
        model.load_state_dict(torch.load(model_path))
        model.to(device_param)
        #if we skip training, we still get a loaded, valid model back, not None
        return model

    # # Define loss function and optimizer
    # criterion = nn.MSELoss()
    # optimizer = optim.Adam(model.parameters(), lr=lr)
    
    model.to(device_param)
    best_val_loss = float('inf')

    # Early stopping setup
    patience = 3  # Stop if validation loss doesn't improve for 3 epochs
    no_improvement = 0
    
    
    for e in range(epochs):
        model.train()
        for batch_idx, (input, target) in enumerate (train_loader):
            print(f"Train Epoch {e}, Batch {batch_idx}, inputs shape {input.shape}, target shape {target.shape}")
            input = input.to(device_param) # clip_name = input
            target = target.to(device_param) # label = target

            optimizer.zero_grad() #clears grads after each batch
            #[batch, num_fames, num_channels, width, height]
            input = input.permute(0, 2, 1, 3, 4) #the order videomae expects
            outputs = model(input) # does a forward pass
            loss = criterion(outputs.logits.squeeze(), target) # calculates the loss b/w the o/p & target
            loss.backward() #calculates the gradients
            optimizer.step() #updates the weights

            if batch_idx % 10 == 0:
                print(f"Train Epoch {e}, Batch {batch_idx}, Loss {loss.item()}")

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_batch_idx, (val_input, val_target) in enumerate(val_loader):
                val_input = val_input.to(device_param)
                val_target = val_target.to(device_param)
                #[batch, num_fames, num_channels, width, height]
                val_input = val_input.permute(0, 2, 1, 3, 4) #the order videomae expects
                
                val_outputs = model(val_input)
                batch_loss = criterion(val_outputs.logits.squeeze(), val_target)
                val_loss += batch_loss.item()

        val_loss = val_loss / len(val_loader)
        print(f"Validation Loss: {val_loss}")

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_path)
            print(f"Model saved to {model_path}")
        else:
            no_improvement += 1
            print(f"No improvement for {no_improvement} epochs")
        # Early stopping check
        if no_improvement >= patience:
            print(f"Early stopping after {e+1} epochs - no improvement for {patience} epochs")
            break
        # Save the model every 5 epochs
        if e % 5 == 0: 
            torch.save(model.state_dict(), model_path)
            print(f"Model saved to {model_path}")

    return model

In [5]:
# if __name__ == "__main__":
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     if device == "cuda":
#         device = torch.device("cuda")
#     else:
#         device = torch.device("cpu")

#     model = build_videomae()
#     train_dataset = CollisionForecastDataset(csv_path="/home/atupulazi/personal_projects/collision-forecast/labels/train_clip_labels.csv", split="train")
#     val_dataset = CollisionForecastDataset(csv_path="/home/atupulazi/personal_projects/collision-forecast/labels/val_clip_labels.csv", split="val" )

#     train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
    
      
#     trained_model = train_model(
#     model=model,
#     optimizer=optimizer,
#     criterion=criterion,
#     device_param=device,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     FORCE_RETRAIN=True,
#     epochs=num_epochs
# )



In [6]:
for inputs, targets in train_loader:
    print("inputs shape:", inputs.shape)
    print("targets shape:", targets.shape)
    break  

NameError: name 'train_loader' is not defined

In [ ]:
# 1. Collect all validation targets
all_targets = []
for _, targets in val_loader:
    all_targets.append(targets)
all_targets = torch.cat(all_targets)  # shape: [num_samples]

# 2. Calculate the mean of all targets
mean_value = all_targets.float().mean()

# 3. Create baseline predictions (all the same value)
baseline_predictions = torch.full_like(all_targets, mean_value.item())
# 4. Calculate MSE loss
mse_loss = torch.nn.functional.mse_loss(baseline_predictions, all_targets.float())
print("Baseline MSE loss:", mse_loss.item())

Baseline MSE loss: 17.969560623168945


In [ ]:
# 🎯 Enhanced Crash Forecast Attention Visualization Pipeline

# ## 🧠 Goal:
# Help users understand **which video frames your model focused on** when predicting a crash,
# with **quantitative metrics** showing contribution percentages and confidence levels.

# ---

# ## ✅ Enhanced Step-by-Step Pipeline

# ### Step 1: Load Preprocessed Video Clip
# - Load the `.npy` file (each one is a 16-frame video clip).
# - This is the same clip used during model inference.

# ### Step 2: Run Model to Predict Crash
# - Send the clip into your trained VideoMAE model.
# - Get the prediction (e.g., "Crash likely").
# - **NEW**: Calculate confidence score from model output.

# ### Step 3: Capture Attention Map from Inside the Model
# - Modify your model to save its **self-attention weights** during inference.
# - Save this as `model.saved_attention`.
# - The attention map shape will be: `[1, 12, 3136, 3136]`

# ### Step 4: Convert Attention → Frame Importance with Metrics
# - Each frame has 196 patches (14x14 grid).
# - Group those patches by frame.
# - Measure how much attention the model gave to each frame's patches.
# - **NEW**: Calculate frame contribution percentages.
# - **NEW**: Calculate overall confidence score.

# ### Step 5: Enhanced Display in Streamlit
# - For each frame:
#   - Show the frame as an image
#   - Show: "Frame X contributed 23% to crash prediction"
#   - Show: "Overall confidence: 87% that this is a crash scenario"
#   - Show: "Attention score: 0.156 (high attention)"
# - Sort by attention score to show most-important frames first.

# ---

# ## 🚀 Enhanced Result
# An interactive Streamlit dashboard that shows:
# - What your model predicted
# - What frames it paid attention to (and how much)
# - **NEW**: Quantitative contribution percentages
# - **NEW**: Confidence scores with explanations
# - Why it made that decision (visually + numerically)

# ---

# ## 📊 Quantitative Metrics Implementation

# ### Frame Contribution Percentage:
# ```python
# def calculate_frame_contribution(attention_weights, frame_idx):
#     """Calculate what percentage this frame contributed to the prediction"""
#     frame_patches = attention_weights[frame_idx * 196:(frame_idx + 1) * 196]
#     total_attention = attention_weights.sum()
#     frame_contribution = frame_patches.sum() / total_attention * 100
#     return f"Frame {frame_idx} contributed {frame_contribution:.1f}% to crash prediction"
# ```

# ### Confidence Score:
# ```python
# def calculate_confidence(model_output):
#     """Calculate confidence from model's raw output"""
#     # For regression: confidence based on how far from decision boundary
#     # For classification: softmax probabilities
#     confidence = abs(model_output - 0.5) * 2 * 100  # Convert to percentage
#     return f"Confidence: {confidence:.1f}% that this is a crash scenario"
# ```

# ### Attention Score:
# ```python
# def calculate_attention_score(frame_attention):
#     """Calculate normalized attention score for this frame"""
#     normalized_score = frame_attention / frame_attention.max()
#     return f"Attention score: {normalized_score:.3f} (high attention)" if normalized_score > 0.5 else f"Attention score: {normalized_score:.3f} (low attention)"
# ```

# ---

# ## 🎁 Bonus Enhancements
# - Helps explain the model's reasoning with numbers
# - Makes your AV crash forecast project interview-ready
# - Shows you're not just running models — you're quantifying their decisions
# - Provides actionable insights for model improvement

# Let me know when you're ready to implement these quantitative metrics!